<a href="https://colab.research.google.com/github/GeraintWong/f20aa_cw2/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("train.csv")
train_df.head()

,Score,Text
0,5,I received this product early from the seller!...
1,5,*****<br />Numi's Collection Assortment Melang...
2,5,"I was very careful not to overcook this pasta,..."
3,5,Buying this multi-pack I was misled by the pic...
4,5,These bars are so good! I loved them warmed up...


In [3]:
train_df['Text_Processed'] = train_df['Text'].str.lower()
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller!...
1,5,*****<br />Numi's Collection Assortment Melang...,*****<br />numi's collection assortment melang...
2,5,"I was very careful not to overcook this pasta,...","i was very careful not to overcook this pasta,..."
3,5,Buying this multi-pack I was misled by the pic...,buying this multi-pack i was misled by the pic...
4,5,These bars are so good! I loved them warmed up...,these bars are so good! i loved them warmed up...


In [4]:
import string
train_df['Text_Processed'] = train_df['Text_Processed'].str.translate(str.maketrans('', '', string.punctuation))
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller ...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",i was very careful not to overcook this pasta ...
3,5,Buying this multi-pack I was misled by the pic...,buying this multipack i was misled by the pict...
4,5,These bars are so good! I loved them warmed up...,these bars are so good i loved them warmed up ...


In [5]:
train_df['Text_Processed'] = train_df['Text_Processed'].str.replace(r'\d+', '', regex=True)
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller ...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",i was very careful not to overcook this pasta ...
3,5,Buying this multi-pack I was misled by the pic...,buying this multipack i was misled by the pict...
4,5,These bars are so good! I loved them warmed up...,these bars are so good i loved them warmed up ...


In [10]:
import nltk
nltk.download('punkt_tab')
print(nltk.data.path)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical

# Preprocessing: Tokenization & Cleaning
train_df['Text_Processed'] = train_df['Text_Processed'].apply(lambda x: ' '.join(word_tokenize(str(x).lower())))

# Splitting dataset
X = train_df['Text_Processed'].values
y = train_df['Score'].values

# One-hot encoding the target labels (Score from 1 to 5)
y = to_categorical(y - 1, num_classes=5)  # Subtracting 1 to align labels from 0 to 4

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization
max_words = 10000  # Vocabulary size
max_len = 100  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

# Building LSTM Model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    SpatialDropout1D(0.2),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # Output layer for 5 classes (1-5)
])

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data=(X_test_pad, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 557s 143ms/step - accuracy: 0.6456 - loss: 1.0649 - val_accuracy: 0.7062 - val_loss: 0.7838
Epoch 2/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 524s 136ms/step - accuracy: 0.7148 - loss: 0.7575 - val_accuracy: 0.7309 - val_loss: 0.7246
Epoch 3/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 522s 135ms/step - accuracy: 0.7405 - loss: 0.6802 - val_accuracy: 0.7375 - val_loss: 0.6930
Epoch 4/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 522s 135ms/step - accuracy: 0.7568 - loss: 0.6326 - val_accuracy: 0.7390 - val_loss: 0.6937
Epoch 5/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 522s 135ms/step - accuracy: 0.7710 - loss: 0.5949 - val_accuracy: 0.7314 - val_loss: 0.7212
Epoch 6/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 522s 135ms/step - accuracy: 0.7848 - loss: 0.5612 - val_accuracy: 0.7350 - val_loss: 0.7251
Epoch 7/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 521s 135ms/step - accuracy: 0.7986 - loss: 0.5265 - val_accuracy: 0.7349 - val_loss: 0.7429
Epoch 8/10
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 521s 135ms/step - ac

In [20]:
test_df = pd.read_csv("test.csv")
test_df.head()

,Id,Text
0,0,Fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000..."


In [21]:
test_df['Text_Processed'] = test_df['Text'].str.lower()
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...","we found this at pf changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamela's almond biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...","<a href=""http://www.amazon.com/gp/product/b000..."


In [24]:
import re
def clean_text_with_html_urls(df, text_column='Text_Processed'):
    """
    Cleans text within a pandas DataFrame column, removing HTML anchor tags and URLs,
    and converting the text to lowercase.

    Args:
        df (pd.DataFrame): The DataFrame containing the text data.
        text_column (str): The name of the column containing the text.

    Returns:
        pd.DataFrame: The DataFrame with cleaned text.
    """

    def remove_html_urls(text):
        """Removes HTML anchor tags containing URLs."""
        html_url_pattern = re.compile(r'<a\s+href="http[^"]*">')
        return html_url_pattern.sub(r'', text)

    def remove_url_remainder(text):
        """Removes the remainder of the anchor tag."""
        remainder_pattern = re.compile(r'</a>')
        return remainder_pattern.sub(r'', text)

    def remove_all_urls(text):
        """Combines to remove both the anchor tag and the url"""
        text = remove_html_urls(text)
        text = remove_url_remainder(text)
        return text

    df[text_column] = df[text_column].astype(str).apply(remove_all_urls) #handle nan and other types
    df[text_column] = df[text_column].str.lower()
    return df


In [25]:
test_df = clean_text_with_html_urls(test_df)
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...","we found this at pf changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamela's almond biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [26]:
test_df['Text_Processed'] = test_df['Text_Processed'].str.translate(str.maketrans('', '', string.punctuation))
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...",we found this at pf changs and it tastes just ...
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamelas almond biscott...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [27]:
test_df['Text_Processed'] = test_df['Text_Processed'].str.replace(r'\d+', '', regex=True)
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...",we found this at pf changs and it tastes just ...
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamelas almond biscott...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [28]:
test_df['Text_Processed'] = test_df['Text_Processed'].apply(lambda x: ' '.join(word_tokenize(str(x).lower())))

In [29]:
X_test_submission = test_df['Text_Processed'].values
tokenizer.fit_on_texts(X_test_submission)

In [30]:
X_test_seq_submission = tokenizer.texts_to_sequences(X_test_submission)

In [31]:
X_test_pad_submission = pad_sequences(X_test_seq_submission, maxlen=max_len, padding="post", truncating="post")

In [35]:
# Predict class probabilities
predictions = model.predict(X_test_pad_submission)

# Convert probabilities to class labels
y_pred = predictions.argmax(axis=1)  # Get class index with highest probability

# If labels start from 1 (not 0), adjust
y_pred += 1  # Convert 0-based indices to 1-based labels
print(y_pred[:10])

3740/3740 ━━━━━━━━━━━━━━━━━━━━ 72s 19ms/step
[4 5 5 5 5 5 5 5 1 5]


In [33]:
sample_submission = pd.read_csv("sampleSubmission.csv")

# Replace the label column with predictions
sample_submission["Score"] = y_pred  # Ensure the column name matches Kaggle's expected output

# Save as submission.csv
sample_submission.to_csv("okok.csv", index=False)